## Hands-on session 2.5 - working with pre-built models

In this session we cover an example of a full workflow for ReMKiT1D. We will generate a partially built wrapper, extend it, and use it to produce physically relevant fluid simulations of a simplified Scrape-Off Layer. 

Demonstrated concepts: 

- Pre-built wrappers 
- Integration for finding equilibria
- Restarting runs
- Time-dependent terms 

In [ ]:
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds 
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn

from simple_fluid import generatorSimpleFluid

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt

### The physical model 

We will look to model a 1D 2-fluid system with electrons and ions. We solve the following set of equations for both species

$$ \frac{\partial n_b}{\partial t} + \nabla (\Gamma_b) = S_b$$

$$ m_b\frac{\partial \Gamma_b}{\partial t} + \nabla (m_b \Gamma_b u_b) + q_b n_b E= R_{bb'} $$

$$ \frac{\partial W_b}{\partial t} + \nabla \left[(W_b + p_b)u_b + q_b \right] + j_b E= Q_b$$

where $S_b$ is the particle source (here due to implied ionization due to recycling), $R_{bb'}$ is the friction due to Coulomb collision between the two species, and $q_b$ is the Braginskii heat flux. $Q_b$ is the energy source due to any external sources and due to electron-ion thermal equilibration.

The electric field $E$ is obtained from what is effectively a current constraint $j_e+j_i = 0$ (by solving a degenerate Ampere-Maxwell equation). 

The boundary conditions are as follows:

- The upstream boundary (x=0) is reflective, meaning that all fluxes are set to 0 there
- The downstream sheath boundary has an outflow Bohm boundary condition on the particles $u_{b,sh} \ge c_s$, where $c_s$ is the ion acoustic speed.
- At the sheath, any outflow is converted into a recycling particle source $S_{rec} \propto \exp(-(L-x)/\lambda_{ion})$ where $\lambda_{ion}$ is an externally set ionization mean free path. The proportionality constant in front of the source is set so that it balances the flux of particles leaving the system - $\Gamma_{sh}$ (calculated from the Bohm condition above). A corresponding energy sink $Q = -\epsilon_{ion} S$ is added to the electron equation, mimicking the cooling of electrons due to ionization, with $\epsilon_{ion}$ being the user-set energy cost per ionization event.
- The energy outflow boundary condition is given as $q_{b,sh} = \gamma_b \Gamma_{sh} k T_{sh}$

The built-in energy source is given by some upstream energy flux $q_h$, spread uniformly over a number of cells upstream.

The initial conditions are specified by setting the upstream and downstream temperatures, as well as the upstream density. The profiles are then generated using the simple Two Point Model.

### Using pre-built wrapper generators 

In order to facilitate code sharing and flexibility, we encourage users to package their wrappers into parameterized generator functions using Python's kwargs feature. An example of a partial wrapper initialization is given in `simple_fluid.py`, and we will use this to generate the starting wrapper for this session

In [ ]:
rk = generatorSimpleFluid(hdf5OutputFolder = "day_2_5",
                          mpiProcsX = 8,
                          Nx = 64, # Number of spatial cells - the grid is uniform
                          domainLength = 10.0, # Length of the domain in meters
                          Tu = 20.0, # Upstream initial temperature in eV
                          Td = 5.0, # Downstream initial temperature in eV
                          nu = 0.8, # Upstream initial density in 1e19m^-3
                          Nh = 1, # heating a single cell - this is the same as just setting an upstream inflow heat flux
                          heatingPower = 5.0, # Effective heating flux in MW/m^2
                          ionMFP = 0.5, # Ionization mfp in meters,
                          ionCost = 20.0 # Ionization cost in eV
                          );

The generator above intentionally does not include time integration setups in order to allow extensions. 

### Time integration for finding equilibria

Let's assume we are only interested in the equilibrium solution of the above system, and do not care too much about the accuracy of time evolution. This is common in practice when looking for good equilibria for launching transients, which we shall examine later in this session. 

A way to reach the equilibrium relatively quickly is to employ an implicit time integration algorithm. In theory, a Backwards Euler step with infinite length is equivalent to solving the equilibrium problem. In practice, the solver has a maximum time step length after which it will (for one reason or another) break. To work around this, the `picardBDEIntegrator` has the `internalStepControl` option, which will instruct the integrator to reduce it's step length when it detects an integration failure, while periodically attempting to return to a single internal step. 

Let's choose a relatively long time step and set the internal step control option.

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["ne", "ni", "Ge_dual", "Gi_dual", "We", "Wi"], internalStepControl=True)

rk.addIntegrator("BE", integrator)

initialTimestep=101.0
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE", bdeStep.dict())

Since we're going for something close to equilibrium, let's ask the simulation to run for a relatively long time of 20000 normalized times (around 1.5ms in this case).

In [ ]:
rk.setTimeTargetTimestepping(20000)
rk.setMinimumIntervalOutput(1000)

We want to be able to restart the simulation, so let's ask the wrapper to keep a checkpoint every 50 steps

In [ ]:
rk.setRestartOptions(save=True, # Save restart data
                     load=False, # Start from initial conditions and not from a saved restart file
                     frequency=50) # Save restart data every 50 steps

### Create config 

To run the above, use the command 

```
mpirun -np 8 /home/stefan/SMijin/ReMKiT1D/build/src/executables/ReMKiT1D/ReMKiT1D | grep -E "Out|Int|Dump"
```

This will trim the output so we can see only the finished integration calls, when variable data is written to file, as well as when restart data is dumped. 

You will notice that the solver speeds up after a slow start.

In [ ]:
rk.writeConfigFile()

### Data analysis 

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=20
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedData

We can have a quick look to see that the simulation is converging

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)
dashboard.fluid2Comparison().show()

### Adding a time-dependent perturbation 

Let's assume we're happy with how close the above simulation is to equilibrium, and that we now wish to perturb the heating power. 

One way of doing this would be changing the input power into the system and restarting the simulation. However, we might want a periodic perturbation. We will see how to add a periodic perturbation to our heating source, but first we must rebuild the wrapper.

In [ ]:
rk = generatorSimpleFluid(hdf5OutputFolder = "day_2_5",
                          mpiProcsX = 8,
                          Nx = 64, # Number of spatial cells - the grid is uniform
                          domainLength = 10.0, # Length of the domain in meters
                          Tu = 20.0, # Upstream initial temperature in eV
                          Td = 5.0, # Downstream initial temperature in eV
                          nu = 0.8, # Upstream initial density in 1e19m^-3
                          Nh = 1, # heating a single cell - this is the same as just setting an upstream inflow heat flux
                          heatingPower = 5.0, # Effective heating flux in MW/m^2
                          ionMFP = 0.5, # Ionization mfp in meters,
                          ionCost = 20.0 # Ionization cost in eV
                          );

### Time signal data

While it is possible to add explicit time dependence by composing different derivation objects, ReMKiT1D offers a built-in way of adding periodic time dependence to matrix terms by using the `TimeSignalData` class. 

The documentation covers the two currently available signal forms (a hat funcion and a cut sine). See the Usage section below

In [ ]:
sc.TimeSignalData?

Let us create a `TimeSignalData` object with a "cutSine" profile, and a $100\mu \text{s}$ period, with the signal active over a 30% of the period, and with a starting delay of $20\mu \text{s}$.

In [ ]:
timeSignal = sc.TimeSignalData(signalType="cutSine",period=1e-4,params=[0.2,0.5],realPeriod=True)

We will use the `simpleSourceTerm` from `common_models` to add a heating perturbation with the above time dependence

In [ ]:
cm.simpleSourceTerm?

Let's add a fast perturbation to the electron heating. The electron energy density variable is "We", and we would like to add a term of the form

$$ \frac{\partial W_e}{\partial t} = Q(x,t) $$

where we would like to have $$ Q(x,t) =   T(t)  X(t) q_{eff}/L_h$$, with $q_{eff}$ being the effective perturbation flux amplitude, $L_h$ the heating length, and $X$ and $T$ dimensionless spatial and temporal profiles.

The `spatialProfile` argument to `simpleSourceTerm` should be set to the normalized value of $X(x)q_{eff}/L_h$. 

This example uses the default normalization, where energy density is normalized to $n_0 eT_0$, and time is normalized to $t_0$. 

We can use the `sk_normalization` module (specifically the `calculateNorms` function) to get the time normalization by first querying the wrapper about its normalization values.

In [ ]:
rk.normalization

In [ ]:
densNorm = rk.normalization["density"]
tempNorm = rk. normalization["eVTemperature"]

In [ ]:
timeNorm=skn.calculateNorms(10.0,1e19,1.0)['time']

Assuming that we want $q_{eff}$ to be in units of MW/$\text{m}^2$ and that $L_h$ is in meters, the equation with normalization taken out will look like 

$$ \frac{n_0 eT_0}{t_0}\frac{\partial W_e}{\partial t} = X(x) T(t) \frac{q_{eff}}{L_h}\times 10^6$$

Moving all of the normalization values to one side, we get that the `spatialProfile` after normalization should be 

$$ \frac{t_0 \times 10^6}{n_0 eT_0} X(x) \frac{q_{eff}}{L_h} $$ 

Assuming we want to heat only the first cell (as in the equilibrium case), we can calculate $L_h$ as

In [ ]:
L = sum(rk.grid.xWidths) # this should be equal to the domain length we set
Nx = len(rk.grid.xWidths)
Lh = L/Nx

We can then create a numpy array for the spatial profile of the source

In [ ]:
elCharge = 1.60218e-19 # electron charge for convenience

q_eff = 10.0 # Effective heating perturbation flux in MW/m^2

spatialProf = np.zeros(Nx)
spatialProf[0] = timeNorm * q_eff * 1e6 /(Lh*densNorm*elCharge*tempNorm)

Finally, we can add a model with the required source term

In [ ]:
newModel = sc.CustomModel("perturbation")

newModel.addTerm("source",cm.simpleSourceTerm(evolvedVar="We",
                                              sourceProfile=spatialProf,
                                              timeSignal=timeSignal
                                              )
                 )

rk.addModel(newModel)

### Time integration resolving the perturbation

We now want to resolve a few periods of the perturbation, while also resolving electron conductive time scales.

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["ne", "ni", "Ge_dual", "Gi_dual", "We", "Wi"], internalStepControl=True)

rk.addIntegrator("BE", integrator)

initialTimestep=0.5
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE", bdeStep.dict())

We use `scalingTimestepController` to scale the time step so we always resolve the collisional time scales

In [ ]:
rk.setTimestepController(sc.scalingTimestepController(reqVarNames=["ne", "Te"],reqVarPowers=[-1.0, 1.5]))

Let's assume that, for whatever reason, we wish to run 3 periods of the perturbation. We can do this by setting a real time target for the integration

In [ ]:
rk.setTimeTargetTimestepping(3e-4,realTimeTarget=True)
rk.setMinimumIntervalOutput(100)

We also set the code to restart from the previous simulation, while resetting the value of the time variable to mimic starting from different initial conditions.

In [ ]:
rk.setRestartOptions(save=False, # Save restart data
                     load=True, # Start from initial conditions and not from a saved restart file
                     resetTime=True) # Reset value of the time variable

It would have also been possible to directly restart from an hdf5 file using the `setHDF5FileInitialData` method of the wrapper. This is beyond the scope of this workshop, but it allows more flexibility since the number of MPI ranks can be changed, and variables can be added or removed from the simulation, which is particularly useful when adding diagnostics to already completed runs.

### Create config 

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=42
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedDataPert = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedDataPert

Let's look at how the perturbation has affected the temperatures

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedDataPert,rk.grid)
dashboard.fluid2Comparison().show()

In [ ]:
dashboard.fluidMultiComparison(["Te","Ti"])

In [ ]:
dashboard.fluidMultiComparison(["Te","Ti"],fixedPosition=True)